# optionm

In [ ]:
//// test

open testing

In [ ]:
open rust_operators

## default_value

In [ ]:
inl default_value d =
    optionm.defaultWith d

In [ ]:
//// test

None
|> default_value 3i32
|> _assert_eq 3i32

assert_eq / actual: 3 / expected: 3


## (/??)

In [ ]:
inl (/??) a b =
    a |> default_value b

In [ ]:
//// test

None /?? 3i32
|> _assert_eq 3i32

assert_eq / actual: 3 / expected: 3


## default_with

In [ ]:
inl default_with fn = function
    | Some x => x
    | None => fn ()

In [ ]:
//// test

None
|> default_with fun () => 3i32
|> _assert_eq 3i32

assert_eq / actual: 3 / expected: 3


## choose

In [ ]:
inl choose fn a b =
    match a, b with
    | Some x, Some y => fn x y |> Some
    | _ => None

In [ ]:
//// test

(Some 2i32, Some 3)
||> choose (+)
|> _assert_eq (Some 5)

assert_eq / actual: US0_0 5 / expected: US0_0 5


## iter

In [ ]:
inl iter fn = function
    | Some x => fn x
    | None => ()

In [ ]:
//// test

inl n = mut 1i32
inl fn =
    fun n' =>
        n <- *n + n'
Some 1i32 |> iter fn
None |> iter fn
*n
|> _assert_eq 2i32

assert_eq / actual: 2 / expected: 2


## option'

In [ ]:
nominal option' t = $"`t option"

## none'

In [ ]:
inl none' forall t. () : option' t =
    $'None'

## some'

In [ ]:
inl some' forall t. (x : t) : option' t =
    $'Some !x '

## default_value'

In [ ]:
inl default_value' forall t. (value : t) (x : option' t) : t =
    $'!x |> Option.defaultValue !value '

## try'

In [ ]:
inl try' forall t. (x : option' t) : t =
    !\\(x, $'"$0?"')

## map

In [ ]:
inl map forall t u. (fn : t -> u) (x : option' t) : option' u =
    inl result : option' u = none' ()
    $'let _!result = ref !result '
    $'match !x with'
    $'| Some x -> ('
    $'(fun () ->'
    $'(fun () ->'
    inl x = dyn $'x'
    x |> fn |> emit_unit
    $')'
    $"|> fun x -> x () |> Some"
    $') () ) | None -> None'
    $"|> fun x -> _!result.Value <- x"
    $'_!result.Value '

## box

In [ ]:
inl box forall t. (x : option t) : option' t =
    // x
    // |> optionm.map some'
    // |> default_with none'
    match x with
    | Some x => some' x
    | None => none' ()

## map'

In [ ]:
inl map' forall t u. (fn : t -> u) (x : option' t) : option' u =
    (!\($'"true; let _result = !x.map(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result"')

## map''

In [ ]:
inl map'' forall t u. (fn : t -> u) (x : option' t) : option' u =
    $'!x |> Option.map !fn '

## unbox

In [ ]:
inl unbox forall t. (x : option' t) : option t =
    x |> map Some |> default_value' None
    // inl some x : option t = Some x
    // inl some = join some
    // inl none : option t = None
    // $'!x |> Option.map !some |> Option.defaultValue !none '

In [ ]:
//// test

inl x = Some 3i32
inl y : option i32 = None
inl x' = x |> box |> unbox
inl y' = y |> box |> map id |> unbox
(x', y') |> _assert_eq (x, y)

assert_eq / actual: struct (US0_0 3, US0_1) / expected: struct (US0_0 3, US0_1)


In [ ]:
//// test
///! rust

inl x = Some 3i32
inl y : option i32 = None
inl x' = x |> box |> unbox
inl y' = y |> box |> map id |> unbox
(x', y') |> _assert_eq' (x, y)

assert_eq' / actual: (US0_0(3), US0_1) / expected: (US0_0(3), US0_1)


## of_obj

In [ ]:
inl of_obj forall t. (x : t) : option' t =
    $'let mutable _!x = None'
    $"#if \!FABLE_COMPILER && \!WASM && \!CONTRACT"
    ((x |> $'Option.ofObj') : option' t) |> emit_unit
    $"#else"
    $'Some !x '
    $"#endif"
    $"|> fun x -> _!x <- Some x"
    $'_!x.Value'

In [ ]:
//// test

null ()
|> of_obj
|> unbox
|> _assert_eq (None : option string)

assert_eq / actual: US0_1 / expected: US0_1


In [ ]:
//// test
///! rust

""
|> of_obj
|> unbox
|> _assert_eq' (Some "")

assert_eq' / actual: US0_0("") / expected: US0_0("")


## flatten

In [ ]:
inl flatten x =
    match x with
    | Some (Some x) => Some x
    | _ => None

## unwrap

In [ ]:
inl unwrap forall t. (x : option' t) : t =
    !\\(x, $'"$0.unwrap()"')

## take

In [ ]:
inl take forall t. (x : option' t) : option' t =
    inl x = join x
    (!\($'"true; let mut !x = !x"') : bool) |> ignore
    !\\(x, $'"Option::take(&mut $0)"')

## ref_take

In [ ]:
inl ref_take forall t. (x : rust.ref' (rust.mut' (option' t))) : option' t =
    !\\(x, $'"Option::take($0)"')

## unwrap_or

In [ ]:
inl unwrap_or forall t. (def : t) (x : option' t) : t =
    !\($'"!x.unwrap_or(!def)"')

## rc_upgrade

In [ ]:
inl rc_upgrade forall t. (x : rust.weak_rc t) : option' (rust.rc t) =
    !\\(x, $'"std::rc::Weak::upgrade(&$0)"')

## rc_into_inner

In [ ]:
inl rc_into_inner forall t. (x : rust.rc t) : option' t =
    !\\(x, $'"std::rc::Rc::into_inner($0)"')

In [ ]:
//// test
///! rust

types ()
rust.new_rc 0i32
|> rc_into_inner
|> unbox
|> _assert_eq' (Some 0i32)

assert_eq' / actual: US0_0(0) / expected: US0_0(0)
